In [1]:
import sys
import re
functions_at = '/proj/modeling/abhi'
sys.path = [f'{functions_at}'] + sys.path
from model_CNN_source import *
tmp_at = '/proj/tmp_data'
dump_at = '/proj/dump'
source_data = '/proj/source_data/Training_Data'
source_data_2 = '/proj/source_data/Testing_Data'
processed_data = '/proj/processed_data'

In [2]:
## trait_data
trait_data = pd.read_csv(f"{source_data}/1_Training_Trait_Data_2014_2021.csv")
trait_data = trait_data[trait_data.Yield_Mg_ha.notnull()]
## meta data
meta_data = pd.read_csv(f"{source_data}/2_Training_Meta_Data_2014_2021_utf_encoded.csv").add_prefix("mt_dta_")
## geno data
geno_data = pd.read_csv(f"{processed_data}/geno_processed.miss.1.mac.1.biallelic.txt").add_prefix("ge_dta_")
geno_data = geno_data.iloc[:, 0:10] # it haas no missing data
## soil data
soil_data = pd.read_csv(f"{source_data}/3_Training_Soil_Data_2015_2021.csv").add_prefix("sl_dta_")
## weather data
weather_data = pd.read_csv(f"{source_data}/4_Training_Weather_Data_2014_2021.csv").add_prefix("wt_dta_")
## EC data
ec_data = pd.read_csv(f"{source_data}/6_Training_EC_Data_2014_2021.csv").add_prefix("ec_dta_")
# testing data to be provided
submission_data = pd.read_csv(f"{source_data_2}/1_Submission_Template_2022.csv")

In [3]:
# remove columns with excess missing data

# pheno data 
pheno_data = purge_excess_missing(trait_data, id_cols = ["Env", "Year", "Field_Location", "Experiment", "Replicate", "Block", "Plot"], plot = False) # the plot shows missing values in columns not in the id_cols list

# meta data
meta_data_purged = purge_excess_missing(meta_data, id_cols = ["mt_dta_Year", "mt_dta_Env", "mt_dta_Experiment_Code"], plot = False)

# soil_data overview
soil_data_purged = purge_excess_missing(soil_data, id_cols = ["sl_dta_Year", "sl_dta_Env"], plot = False)

# weather data reshape
weather_data.groupby(["wt_dta_Env"])["wt_dta_Env"].count()
weather_data["wt_dta_moth_dy"] = weather_data["wt_dta_Date"].astype(str).str[4:8]
weather_data_wide = weather_data.pivot(index="wt_dta_Env", columns="wt_dta_moth_dy", values=weather_data.columns.tolist()[2:-1])
weather_data_wide.columns = ['_'.join(map(str, x)) for x in weather_data_wide.columns] #212 * (16*366)
weather_data_wide_df = weather_data_wide.reset_index()
weather_data_purged = purge_excess_missing(weather_data_wide_df, id_cols = ["wt_dta_Env"], plot = False)

# ec data
ec_data_purged = purge_excess_missing(ec_data, id_cols = ["ec_dta_Env"], plot = False)

# remove objects to free memory
#del trait_data
#del meta_data
#del soil_data
#del weather_data
#del weather_data_wide
#del ec_data

In [4]:
# remove those environments which are completele missing soil, weather, or ec data
purge_1 = pheno_data[pheno_data.Env.isin(weather_data_purged.wt_dta_Env.unique())]
purge_2 = purge_1[purge_1.Env.isin(ec_data_purged.ec_dta_Env.unique())]
purge_3 = purge_2[purge_2.Env.isin(soil_data_purged.sl_dta_Env.unique())]
purge_4 = purge_3[purge_3.Hybrid.isin(geno_data.ge_dta_Hybrid.unique())]

print(f'pheno_data has {pheno_data.Env.nunique()} environments, purge 1 removes {pheno_data.Env.nunique() - purge_1.Env.nunique()}, purge 2 removes {purge_1.Env.nunique() - purge_2.Env.nunique()}, purge 3 removes {purge_2.Env.nunique() - purge_3.Env.nunique()}, thus the final data has {purge_3.Env.nunique()} environments')
print(f'purge 4 removed {purge_3.shape[0] - purge_4.shape[0]} rows due to non availability of genotypic data. Data now has {purge_4.Env.nunique()} environments and {purge_4.Hybrid.nunique()} unique genotypes in {purge_3.shape[0]} rows')

# we can pull env and soil data from public data bases to put them back in but lets start with a conservative set.

# join data together for the environments with all data available
merged_data = purge_4.merge(soil_data_purged, how="left", left_on="Env", right_on="sl_dta_Env").merge(weather_data_purged, how="left", left_on="Env", right_on="wt_dta_Env").merge(ec_data_purged, how="left", left_on="Env", right_on="ec_dta_Env")

pheno_data has 217 environments, purge 1 removes 5, purge 2 removes 52, purge 3 removes 51, thus the final data has 109 environments
purge 4 removed 1553 rows due to non availability of genotypic data. Data now has 109 environments and 3827 unique genotypes in 69360 rows


In [5]:
# Check missing data freuency for all explanotory variables in each env
id_cols = ["Env"]
data_missing = merged_data[merged_data.columns.difference(id_cols)].isna()
data_missing = pd.concat([merged_data.loc[:, id_cols], data_missing], axis = 1)
data_missing_overview = data_missing.groupby(id_cols[0]).sum()/merged_data.groupby("Env").count() # gives propotions of missing values per env for a given variable
#for i in range(data_missing_overview.shape[0]):
#    print(pd.cut(x=data_missing_overview.iloc[i, :].values.tolist(), bins=[-0.1, 0, 0.20, 0.40, 0.60, 0.80, 1]).unique()) # mostly less than 20 percent missing epr env. so i impute these
data_missing_overview.to_csv(f"{processed_data}/missign_overview_per_env.csv", index = False)

In [6]:
# define coltypes to impute
coltypes = merged_data[merged_data.columns.difference(pheno_data.columns)].dtypes.values.astype("str")
data_select = []
for Type in coltypes:
    if "float64" in Type:
        data_select.append(True)
    elif "int64" in Type :
        data_select.append(True)
    else:
        data_select.append(False)
data_nonselect = [not x for x in data_select]
col_select = merged_data[merged_data.columns.difference(pheno_data.columns)].columns[data_select].tolist()
col_select_with_env = ["Env"]+col_select 
col_nonselect = merged_data[merged_data.columns.difference(pheno_data.columns)].columns[data_nonselect].tolist()
col_nonselect = pheno_data.columns.tolist() + col_nonselect # will not be imputed

merged_data_raw = merged_data.loc[:, col_nonselect]
merged_data_to_impute = merged_data.loc[:, col_select_with_env]

In [7]:
# impute 
store_exceptions = []
merged_data_imputed = merged_data_to_impute.iloc[:, 0:1]
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=RuntimeWarning)
    for i, col in enumerate(col_select):
        if i%1000 == 0:
            print(f'finished for {i}')
        val = None
        val = merged_data_to_impute.groupby(['Env'])[col].apply(lambda x: x.fillna(x.median()))
        if val.isnull().sum() == 0:
            merged_data_imputed = pd.concat([merged_data_imputed, val], axis = 1) # store only those columns which can be imputed
        else:
            store_exceptions.append(col)
            print(f'{col} has issues with median value calculation. check it manually')
# sanity check
data_missing = merged_data_imputed.isna().sum()/merged_data_imputed.shape[0]
data_missing[data_missing != 0]
final_imputed_data = pd.concat([merged_data_raw, merged_data_imputed[merged_data_imputed.columns.difference(["Env"])]], axis = 1)

finished for 0
sl_dta_% Clay has issues with median value calculation. check it manually
sl_dta_% Sand has issues with median value calculation. check it manually
sl_dta_% Silt has issues with median value calculation. check it manually
sl_dta_%Ca Sat has issues with median value calculation. check it manually
sl_dta_%H Sat has issues with median value calculation. check it manually
sl_dta_%K Sat has issues with median value calculation. check it manually
sl_dta_%Mg Sat has issues with median value calculation. check it manually
sl_dta_%Na Sat has issues with median value calculation. check it manually
sl_dta_1:1 S Salts mmho/cm has issues with median value calculation. check it manually
sl_dta_1:1 Soil pH has issues with median value calculation. check it manually
sl_dta_CEC/Sum of Cations me/100g has issues with median value calculation. check it manually
sl_dta_Calcium ppm Ca has issues with median value calculation. check it manually
sl_dta_E Depth has issues with median value calc

In [8]:
final_imputed_data.to_csv(f"{processed_data}/combined_mat.csv", index = False)

In [9]:
# assuming that we arrive on object called final_data after filtereing and if needed some imputation

final_data = final_imputed_data.loc[:, ['Env', 'Year', "Hybrid"]]

In [10]:
# create train test splits and save them V_1: true cv without a val set. you can sample 10 percent from the trian set to be val set

fold = 10
runs = 100
run = 0
out_dict_1 = {}
while run < runs:
    out_dict_2 = {}
    if run > 0:
        del kf
    kf = KFold(n_splits=fold, random_state=40+(20*run), shuffle=True)
    fold = 0
    for train_index, test_index in kf.split(final_data.index):
        in_dict = {}
        in_dict["train"] = train_index.tolist() # does not work since the train set looses all info 
        in_dict["test"] = test_index.tolist()
        out_dict_2[fold] = in_dict
        fold += 1
    out_dict_1[run] = out_dict_2
    run += 1
# write json file
# write_json(data = out_dict_1, path = f"{processed_data}/train_test_split.json")

In [12]:
# create train test splits and save them V_2. Custom cv with val set

sets={}
hld_year_geno=[]

total_years = final_data.Year.unique()
for year in total_years:
    for rep in range(0,10):
        # test data
        test_data = final_data[final_data.Year.isin([year])]
        test_geno_total = test_data.Hybrid.unique().tolist()
        test_geno_sub = random.sample(test_geno_total, int(len(test_geno_total)*0.2))
        test_set = test_data[test_data.Hybrid.isin(test_geno_sub)]
        
        # val data
        train_data = final_data[~final_data.Year.isin([year])]
        train_data_sub = train_data[~train_data.Hybrid.isin(test_geno_sub)]
        
        val_idx = sorted(random.sample(train_data_sub.index.tolist(), int(0.1*len(train_data_sub.index.tolist()))))
        val_set = train_data_sub.loc[val_idx].copy()
        
        # train_data
        train_set = train_data_sub[~train_data_sub.index.isin(val_idx)]
        
        # sanity checks
        if len(train_set[train_set["Year"].isin(test_set["Year"].unique().tolist())]) !=0:
            print("CONTAMINATED SETS: Year")
        if len(train_set[train_set["Hybrid"].isin(test_set["Hybrid"].unique().tolist())]) !=0:
            print("CONTAMINATED SETS: Genotype")
            
        # produce output
        sets[str(year)+"@"+str(rep)]={"train":train_set.index.tolist(),
                 "val":val_set.index.tolist(),
                 "test":test_set.index.tolist()}
                #record data for diagnostic purposes
        hld_year_geno.append([year, rep, len(test_set["Hybrid"].unique())/len(test_geno_total),
                              len(train_set["Hybrid"].unique())/len(final_data["Hybrid"].unique()),
                              len(train_set), len(val_set), len(test_set)])
print(len(sets))
# write json file
write_json(data = sets, path = f"{processed_data}/train_test_split_v2.json") # can think of saving the hld_year_geno also if needed

70
Done


In [4]:
## submission data
training_data = pd.read_csv(f"{processed_data}/combined_mat.csv")
training_data['type'] = "train"

## testing data to be provided
submission_data = pd.read_csv(f"{source_data_2}/1_Submission_Template_2022.csv")
submission_data['type'] = "submission"

## geno data
geno_data = pd.read_csv(f"{processed_data}/geno_processed.miss.1.mac.1.biallelic.txt").add_prefix("ge_dta_")
geno_data = geno_data.iloc[:, 0:10] # it haas no missing data

## soil data
soil_data_sub = pd.read_csv(f"{source_data_2}/3_Testing_Soil_Data_2022.csv").add_prefix("sl_dta_")
## weather data
weather_data_sub = pd.read_csv(f"{source_data_2}/4_Testing_Weather_Data_2022.csv").add_prefix("wt_dta_")
## EC data
ec_data_sub = pd.read_csv(f"{source_data_2}/6_Testing_EC_Data_2022.csv").add_prefix("ec_dta_")


/tmp/ipykernel_712/1384639663.py:2: DtypeWarning: Columns (24,28) have mixed types. Specify dtype option on import or set low_memory=False.
  training_data = pd.read_csv(f"{processed_data}/combined_mat.csv")


In [5]:
## genertate submission_data in the same column format as training data
# remove those environments which are completele missing soil, weather, or ec data

# soil_data overview
soil_data_sub_purged = purge_excess_missing(soil_data_sub, id_cols = ["sl_dta_Year", "sl_dta_Env"], plot = False)

# weather data reshape
weather_data_sub.groupby(["wt_dta_Env"])["wt_dta_Env"].count() # 314 days instead of 365 days
weather_data_sub["wt_dta_moth_dy"] = weather_data_sub["wt_dta_Date"].astype(str).str[4:8]
weather_data_sub_wide = weather_data_sub.pivot(index="wt_dta_Env", columns="wt_dta_moth_dy", values=weather_data_sub.columns.tolist()[2:-1])
weather_data_sub_wide.columns = ['_'.join(map(str, x)) for x in weather_data_sub_wide.columns] #212 * (16*366)
weather_data_sub_wide_df = weather_data_sub_wide.reset_index()
weather_data_sub_purged = purge_excess_missing(weather_data_sub_wide_df, id_cols = ["wt_dta_Env"], plot = False)

# ec data
ec_data_sub_purged = purge_excess_missing(ec_data_sub, id_cols = ["ec_dta_Env"], plot = False)

In [6]:
# remove those environments which are completele missing soil, weather, or ec data
purge_sub_1 = submission_data[submission_data.Env.isin(weather_data_sub_purged.wt_dta_Env.unique())]
purge_sub_2 = purge_sub_1[purge_sub_1.Env.isin(ec_data_sub_purged.ec_dta_Env.unique())]
purge_sub_3 = purge_sub_2[purge_sub_2.Env.isin(soil_data_sub_purged.sl_dta_Env.unique())]
purge_sub_4 = purge_sub_3[purge_sub_3.Hybrid.isin(geno_data.ge_dta_Hybrid.unique())]

purge_sub_4.groupby('Env').count().shape # has data for only 20 out of 26 or so envt.

merged_data_sub = purge_sub_4.merge(soil_data_sub_purged, how="left", left_on="Env", right_on="sl_dta_Env").merge(weather_data_sub_purged, how="left", left_on="Env", right_on="wt_dta_Env").merge(ec_data_sub_purged, how="left", left_on="Env", right_on="ec_dta_Env")

In [7]:
## whats missing
miss_cols = {}
miss_cols['wc'] = weather_data_purged.columns[~weather_data_purged.columns.isin(weather_data_sub_purged.columns)]
miss_cols['ec'] = ec_data_purged.columns[~ec_data_purged.columns.isin(ec_data_sub_purged.columns)]
miss_cols['sl'] = soil_data_purged.columns[~soil_data_purged.columns.isin(soil_data_sub_purged.columns)]
miss_cols # 2147 missing only for weather data
info_missing = pd.DataFrame({"days" : [int(f'2022{x[-4:]}') for x in miss_cols['wc']],
                            "variable" : [x[0:-5] for x in miss_cols['wc']]})
vars_miss = info_missing.variable.values.tolist()
info_missing.groupby(['variable']).count() # different number of days are missing for different variables

# more details
#to_check_var = np.unique(vars_miss).tolist()[1]
#respective_days = info_missing.loc[info_missing.variable == to_check_var, "days"].values.tolist()
#weather_data_sub.loc[weather_data_sub.wt_dta_Date.isin(respective_days), to_check_var] # some days are missing and others have completely missing data. I stop here to investigate this

,days
variable,
wt_dta_ALLSKY_SFC_PAR_TOT,305
wt_dta_ALLSKY_SFC_SW_DNI,305
wt_dta_ALLSKY_SFC_SW_DWN,62
wt_dta_GWETPROF,305
wt_dta_GWETROOT,305
wt_dta_GWETTOP,305
wt_dta_PRECTOTCORR,56
wt_dta_PS,56
wt_dta_QV2M,56


In [8]:
missing_overview = merged_data_sub.isna().sum()
columns_with_na = missing_overview[missing_overview > 0].index.tolist()[1:] # only few columns. i will just remove them except the first one
merged_data_sub_no_missing = merged_data_sub.loc[:, ~merged_data_sub.columns.isin(columns_with_na)]
training_data_filtered = training_data.loc[:, training_data.columns.isin(merged_data_sub_no_missing.columns)]
really_final_data = pd.concat([training_data_filtered, merged_data_sub_no_missing]) # loose a lot of columns which are not there in submission data

In [9]:
really_final_data.to_csv(f"{processed_data}/combined_mat_w_o_BLUEs_final.csv", index = False)